In [2]:
import pandas as pd
import numpy as np
import joblib
import optuna

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_squared_error

# CSV faylni o'qish
file_path = r"C:\Users\Rasulbek907\Desktop\LifeSpan_Peoples\Data\Feature_selection\Filtered_Features.csv"
df = pd.read_csv(file_path)

# Features va target
X = df.drop(columns=['life_span'])
y = df['life_span']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Optuna objective function
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 2, 10),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'max_features': trial.suggest_categorical('max_features', ['sqrt','log2', None])
    }
    model = GradientBoostingRegressor(**params, random_state=42)
    score = cross_val_score(model, X_train, y_train, cv=5, scoring='r2').mean()
    return score

# Optuna study yaratish
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)  # n_trials ni oshirib tuning intensivligini oshirish mumkin

print("Eng yaxshi hyperparameterlar:", study.best_params)

# Eng yaxshi modelni yaratish
best_params = study.best_params
best_model = GradientBoostingRegressor(**best_params, random_state=42)
best_model.fit(X_train, y_train)

# Test natijasini tekshirish
y_pred = best_model.predict(X_test)
print(f"R2: {r2_score(y_test, y_pred):.4f}, RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.4f}")

# Modelni saqlash
save_path = r"C:\Users\Rasulbek907\Desktop\LifeSpan_Peoples\Models\best_gradient_boosting_model.pkl"
joblib.dump(best_model, save_path)
print(f"Eng yaxshi Gradient Boosting modeli saqlandi: {save_path}")


[I 2025-11-29 12:59:15,346] A new study created in memory with name: no-name-e15d967c-512a-4302-b7ed-915a63982437


[I 2025-11-29 12:59:17,277] Trial 0 finished with value: 0.26745041157211097 and parameters: {'n_estimators': 219, 'learning_rate': 0.1495663505206393, 'max_depth': 9, 'min_samples_split': 16, 'min_samples_leaf': 8, 'subsample': 0.5633964758966725, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.26745041157211097.
[I 2025-11-29 12:59:35,680] Trial 1 finished with value: 0.41911093002973854 and parameters: {'n_estimators': 495, 'learning_rate': 0.051016672956433244, 'max_depth': 9, 'min_samples_split': 19, 'min_samples_leaf': 12, 'subsample': 0.7453998442488461, 'max_features': None}. Best is trial 1 with value: 0.41911093002973854.
[I 2025-11-29 12:59:41,643] Trial 2 finished with value: -0.4776456287915044 and parameters: {'n_estimators': 199, 'learning_rate': 0.03984461780962504, 'max_depth': 7, 'min_samples_split': 17, 'min_samples_leaf': 4, 'subsample': 0.6807793275416278, 'max_features': None}. Best is trial 1 with value: 0.41911093002973854.
[I 2025-11-29 12:59:42,902] Tri

Eng yaxshi hyperparameterlar: {'n_estimators': 309, 'learning_rate': 0.20072753134389323, 'max_depth': 8, 'min_samples_split': 18, 'min_samples_leaf': 15, 'subsample': 0.7213574506759637, 'max_features': None}
R2: 0.6144, RMSE: 47.4846
Eng yaxshi Gradient Boosting modeli saqlandi: C:\Users\Rasulbek907\Desktop\LifeSpan_Peoples\Models\best_gradient_boosting_model.pkl
